In [8]:
%pip install osmnx
%pip install folium
%pip install geopy
%pip install scikit-learn
import osmnx as ox
import networkx as nx
import folium
import pyproj
from geopy.geocoders import Nominatim
from IPython.display import display
from shapely.geometry import LineString
from shapely.ops import transform

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Initialize geolocator
geolocator = Nominatim(user_agent="my_geo_app")

# Function to get latitude & longitude for a location
def get_location(city):
    location = geolocator.geocode(city)
    return (location.latitude, location.longitude) if location else None

# Take user input for source and destination
source = input("Enter starting location: ")
destination = input("Enter destination: ")

# Get coordinates
source_coords = get_location(source)
destination_coords = get_location(destination)

if not source_coords or not destination_coords:
    print("❌ Could not fetch coordinates for one or both locations.")
    exit()


lat_center = (source_coords[0] + destination_coords[0]) / 2
lon_center = (source_coords[1] + destination_coords[1]) / 2
G = ox.graph_from_point((lat_center, lon_center), dist=5000, network_type='drive')

In [10]:
orig_node = ox.distance.nearest_nodes(G, X=source_coords[1], Y=source_coords[0])
dest_node = ox.distance.nearest_nodes(G, X=destination_coords[1], Y=destination_coords[0])

# Shortest route
shortest_route = nx.shortest_path(G, orig_node, dest_node, weight="length")

# Helper to compute path distance
def compute_path_info(path):
    edges = list(zip(path[:-1], path[1:]))
    total_length_m = sum(G[u][v][0].get("length", 0) for u, v in edges)
    return total_length_m / 1000  # km

# Second shortest path (remove 1st edge)
G_temp = G.copy()
second_route = None
if len(shortest_route) > 1:
    edge_to_remove = (shortest_route[0], shortest_route[1])
    if G_temp.has_edge(*edge_to_remove):
        G_temp.remove_edge(*edge_to_remove)
    try:
        second_route = nx.shortest_path(G_temp, orig_node, dest_node, weight="length")
    except nx.NetworkXNoPath:
        pass

# Ask for fuel price
try:
    fuel_price_per_km = float(input("Enter fuel price per km: "))
except:
    fuel_price_per_km = 5

# Choose transport mode
print("Choose your transport mode:")
print("1. Car\n2. Bus\n3. Two-Wheeler\n4. Walking")
mode = input("Enter option number (1-4): ")

mode_speed_map = {
    "1": 60,  # Car
    "2": 40,  # Bus
    "3": 50,  # Two-Wheeler
    "4": 5    # Walking
}

avg_speed = mode_speed_map.get(mode, 60)

# Allow user to override speed
try:
    override = input(f"Default speed for selected mode is {avg_speed} km/h. Enter custom speed or press Enter to use default: ")
    if override.strip():
        override_speed = float(override)
        if override_speed > 0:
            avg_speed = override_speed
except:
    pass

# Route stats
def route_stats(route, label, color):
    coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    distance_km = compute_path_info(route)
    time = distance_km / avg_speed
    cost = distance_km * fuel_price_per_km
    return {
        "label": label,
        "coords": coords,
        "distance": round(distance_km, 2),
        "hours": int(time),
        "minutes": int((time - int(time)) * 60),
        "cost": round(cost, 2),
        "color": color
    }

# Offset 2nd route
def offset_coords(coords, offset_meters):
    if len(coords) < 2:
        return coords
    proj = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform
    back_proj = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True).transform
    line = LineString([(lon, lat) for lat, lon in coords])
    line_proj = transform(proj, line)
    offset_line = line_proj.parallel_offset(offset_meters, 'left', join_style=2)
    offset_back = transform(back_proj, offset_line)
    return [(lat, lon) for lon, lat in offset_back.coords]

# Build path info
paths = [route_stats(shortest_route, "Shortest Path", "green")]
if second_route:
    second = route_stats(second_route, "Second Shortest Path", "red")
    second["coords"] = offset_coords(second["coords"], 6)
    paths.append(second)

# Print route info
for p in paths:
    print(f"\n🚗 {p['label']}")
    print("📏 Distance:", p["distance"], "km")
    print("⛽ Fuel Cost:", p["cost"])
    print("⏳ Time: {0}h {1}m".format(p["hours"], p["minutes"]))

# Map setup
m = folium.Map(location=source_coords, zoom_start=13)
folium.Marker(source_coords, popup="Start: " + source, icon=folium.Icon(color="green")).add_to(m)
folium.Marker(destination_coords, popup="End: " + destination, icon=folium.Icon(color="red")).add_to(m)

# Red route first
red_path = next((p for p in paths if p["color"] == "red"), None)
if red_path:
    folium.PolyLine(locations=red_path["coords"], color="red", weight=6, opacity=0.9, tooltip=red_path["label"]).add_to(m)

# Green on top
green_path = next((p for p in paths if p["color"] == "green"), None)
if green_path:
    folium.PolyLine(locations=green_path["coords"], color="green", weight=6, opacity=0.9, tooltip=green_path["label"]).add_to(m)

# Extend green to actual start and end
if source_coords != green_path["coords"][0]:
    folium.PolyLine([source_coords, green_path["coords"][0]], color="green", weight=6).add_to(m)
if destination_coords != green_path["coords"][-1]:
    folium.PolyLine([green_path["coords"][-1], destination_coords], color="green", weight=6).add_to(m)

# Extend red path if exists
if red_path:
    if source_coords != red_path["coords"][0]:
        folium.PolyLine([source_coords, red_path["coords"][0]], color="red", weight=6).add_to(m)
    if destination_coords != red_path["coords"][-1]:
        folium.PolyLine([red_path["coords"][-1], destination_coords], color="red", weight=6).add_to(m)

# Show map
m

Choose your transport mode:
1. Car
2. Bus
3. Two-Wheeler
4. Walking

🚗 Shortest Path
📏 Distance: 17.39 km
⛽ Fuel Cost: 1600.05
⏳ Time: 0h 13m
